<a href="https://colab.research.google.com/github/wolfzxcv/ml-examples/blob/master/TSP_using_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Key Components of a Genetic Algorithm**

* Selection: Choosing individuals from the population to create offspring.
* Crossover: Combining parts of two individuals to create a new individual.
* Mutation: Randomly changes genes to introduce diversity.

**Bonus**
* Elitism: Retains top individuals in the next generation to preserve the best solutions.

In [10]:
from random import randint, choice

# Number of cities in TSP
V = 5

# Names of the 5 cities
GENES = "ABCDE"

# Initial population size (potential solutions) for the algorithm
POP_SIZE = 10

# Max number of generations
max_gen = 20

In [11]:
# individual solution (a possible path)
class Individual:
    def __init__(self, gnome, fitness):
        # A string representing the path (e.g., "01234")
        self.gnome = gnome
        # The total distance of the path
        self.fitness = fitness


# Generate a random number between start and end-1
def rand_num(start, end):
    return randint(start, end - 1)  # Adjust for Python's zero-based indexing


# Generates a valid initial path (gnome) starting and ending at the first city
def create_gnome():
    gnome = str(choice(range(V)))
    while len(gnome) < V:
        temp = rand_num(0, V)
        if str(temp) not in gnome:
            gnome += str(temp)
    return gnome


# Perform crossover between two parents to create two offspring
def crossover(parent1, parent2):
    # Get two random crossover points
    startGene = rand_num(1, V)
    endGene = rand_num(1, V)

    # Ensure startGene is less than endGene
    if startGene > endGene:
        startGene, endGene = endGene, startGene

    # Create empty children
    child1 = [-1] * V
    child2 = [-1] * V

    # Copy the selected segment from parent1 to child1
    for i in range(startGene, endGene):
        child1[i] = parent1.gnome[i]

    # Copy the selected segment from parent2 to child2
    for i in range(startGene, endGene):
        child2[i] = parent2.gnome[i]

    # Fill the remaining positions with genes from the other parent
    fill_remaining_genes(parent2.gnome, child1, startGene, endGene)
    fill_remaining_genes(parent1.gnome, child2, startGene, endGene)

    return ''.join(child1), ''.join(child2)

def fill_remaining_genes(parent, child, start, end):
    current_pos = end
    for gene in parent:
        if gene not in child:
            if current_pos == V:
                current_pos = 0
            if current_pos == start:
                current_pos = end
            child[current_pos] = gene
            current_pos += 1


# Mutates a gnome by swapping two random cities in the path
def mutated_gene(gnome):
    gnome = list(gnome)
    while True:
        r = rand_num(1, V)
        r1 = rand_num(1, V)
        if r1 != r:
            temp = gnome[r]
            gnome[r] = gnome[r1]
            gnome[r1] = temp
            break
    return ''.join(gnome)


# Calculates the fitness of a gnome by summing the distances between continuous cities in the path
def calculate_fitness(gnome):
    # A very high value to represent no direct path between cities
    INT_MAX = 2147483647

    # distance_matrix represents the distances between cities
    # The fitness value is the total distance traveled
    # Summing up the distances between continuous cities along the path
    distance_matrix = [
        [0, 2, INT_MAX, 12, 5],
        [2, 0, 4, 8, INT_MAX],
        [INT_MAX, 4, 0, 3, 3],
        [12, 8, 3, 0, 10],
        [5, INT_MAX, 3, 10, 0],
    ]
    fitness = 0
    for i in range(V - 1):
        fitness += distance_matrix[int(gnome[i])][int(gnome[i + 1])]
        # Add the distance from the last city back to the starting city
        fitness += distance_matrix[int(gnome[-1])][int(gnome[0])]
    return fitness

In [12]:
# Main function for TSP problem.
def tsp_util():
    global max_gen  # Declare max_gen as global

    # A list to store the population of individuals
    population = []

    # Current generation number
    gen = 1

    print("Initial population: \nGNOME FITNESS VALUE")
    # Populating the GNOME pool.
    for i in range(POP_SIZE):
        gnome = create_gnome()
        fitness = calculate_fitness(gnome)
        print(gnome, fitness)
        population.append(Individual(gnome, fitness))

    # Iteration to perform
    while gen <= max_gen:
      population.sort(key=lambda x: x.fitness)
      new_population = []

      # Elitism: Carry over the best individuals to the next generation
      elites_count = int(POP_SIZE * 0.5)  # Keep top 50% of the population as elites
      elites = population[:elites_count]
      new_population.extend(elites)

      # Selection, Crossover, and Mutation
      for _ in range((POP_SIZE - elites_count) // 2):
          parent1 = choice(population)
          parent2 = choice(population)
          child1_gnome, child2_gnome = crossover(parent1, parent2)
          mutated_child1_gnome = mutated_gene(child1_gnome)
          mutated_child2_gnome = mutated_gene(child2_gnome)
          new_fitness1 = calculate_fitness(mutated_child1_gnome)
          new_fitness2 = calculate_fitness(mutated_child2_gnome)
          new_population.append(Individual(mutated_child1_gnome, new_fitness1))
          new_population.append(Individual(mutated_child2_gnome, new_fitness2))

      population = new_population

      print(f"\nGeneration {gen}")
      print("GNOME FITNESS VALUE")

      for ind in population:
          print(f"{ind.gnome} {ind.fitness}")

      gen += 1

    # Handle result
    # Find the minimum fitness value
    min_fitness = min(individual.fitness for individual in population)

    # Find all individuals with the minimum fitness value
    best_individuals = [individual for individual in population if individual.fitness == min_fitness]

    # Print the smallest fitness value (shortest path)
    print(f"\nShortest path (smallest fitness value in the last iteration): {min_fitness}")

    # Decode the gnome (visiting order) and print the unique solutions
    unique_results = set()

    for best_individual in best_individuals:
        best_gnome = best_individual.gnome
        result = "".join([GENES[int(city)] for city in best_gnome][::-1])  # List comprehension with reverse
        visiting_order = f"{result}{result[0]}"  # Add starting city at the end
        if visiting_order not in unique_results:
            unique_results.add(visiting_order)
            print(f"Visiting order (city names): {visiting_order}")


if __name__ == "__main__":
    tsp_util()

Initial population: 
GNOME FITNESS VALUE
12430 37
40231 10737418251
43210 39
34201 2147483694
14302 4294967332
41032 2147483676
21430 10737418261
21430 10737418261
21043 33
14023 4294967334

Generation 1
GNOME FITNESS VALUE
21043 33
12430 37
43210 39
41032 2147483676
34201 2147483694
32014 4294967339
43012 40
40132 30
12304 8589934612

Generation 2
GNOME FITNESS VALUE
40132 30
21043 33
12430 37
43210 39
43012 40
24013 30
01324 36
10423 45
12340 30

Generation 3
GNOME FITNESS VALUE
40132 30
24013 30
12340 30
21043 33
01324 36
10342 43
34021 2147483698
01423 2147483703
04321 30

Generation 4
GNOME FITNESS VALUE
40132 30
24013 30
12340 30
04321 30
21043 33
02341 4294967315
21430 10737418261
13240 27
12430 37

Generation 5
GNOME FITNESS VALUE
13240 27
40132 30
24013 30
12340 30
04321 30
14032 2147483683
32041 4294967334
24310 8589934611
43021 10737418261

Generation 6
GNOME FITNESS VALUE
13240 27
40132 30
24013 30
12340 30
04321 30
24310 8589934611
24103 2147483676
43120 2147483689
01234 3